# Chinese Syllable Count Generator Working Draft Notes
This document details the process of experimentation related to the construction of the Chinese Syllable Count Generator function. The function takes romanized Chinese text of any size or word count, evaluates the number of syllables based on the supplied Latin characters, and outputs a list of integers representing the syllables of each word.

The initial stages of development tests three methods of storing data for optimized processing comparison: Python list, NumPy Array, and Pandas DataFrame.

In [1]:
import pandas as pd
import numpy as np

The romanization of Chinese text is based on structured systems with accepted combinations of "initials" (the initial sound of a syllable, e.g., the first consonant) and "finals" (the ending sound of a syllable, e.g., vowels that sometimes end with a consonant). For example, one syllable in pinyin can be as small as one letter and as large as six letters. Therefore, detecting a syllable can occur in one of two ways:
1. A sequence of letters compared to a list of accepted pinyin syllables. If the sequence appears in the list, or if the sequence is part of a longer accepted term ("tia" by itself is not an accepted syllable, however "tiao" and "tian" are), then an additional letter from the pinyin can be added to the sequence and reevaluated. If the sequence does not appear in the list and is not part of a longer accepted term, then the last character of the sequence becomes the first character of the next syllable, and the syllable count incremented.
2. A sequence of letters is processed incrementally with the initials identified first, and then the finals after the appearance of the first vowel. The identified initial and final values act as indexes to a table (either a NumPy Array or Pandas DataFrame) of boolean values that evaluate the initial and final pairing. If the table returns False, then the final is compared to a list of other final values to determine whether it can be part of a more extended final. If it is, then the function continues onto the next incremented letter. Otherwise, the function determines that the current letter is the start of the next syllable and increments the number of syllables for that word.

Given these possibilities, two metrics need to be considered to settle on the proper method for the evaluation function: speed and size. I hypothesize that comparing a sequence of letters to a list of terms can be faster; however, the list itself may be excessively large. Evaluating sequences by their combination of initials and finals, however, may take longer but results in a much smaller package. The following exercises test each method on a smaller scale to get a general sense of the speed vs. size issue.

## Setup for Testing

The code below imports two CSV files. The first contains the list of acceptable pinyin syllables (e.g. "yang," "wei"). The second contains an array of 1 values with a column header of initials and row headers with finals. Empty values convert to 0, and both 1 and 0 values convert to the boolean values True and False, respectively in their array and DataFrame forms.

In [287]:
import csv
import sys

## LIST ##
# Import list of all pinyin combinations
smList = []
with open("data/pinyinList.csv") as f:
    r = csv.reader(f)
    for row in r:
        smList += row
        
## NUMPY/DATAFRAME ##
# List of initials, finals for Numpy evaluation
PY = np.genfromtxt('data/pinyinDF.csv', delimiter=',', dtype=str)
initList = list(PY[:,0][1:])
finList = list(PY[0][1:])

# Import Numpy array from CSV
smAr = PY[1:][:,1:] == '1'

# Import dataframe from CSV 
smDF = pd.read_csv('data/pinyinDF.csv', header=0, index_col=0, dtype={'INDEX':str}
                  ).fillna(0).astype('bool')

# Print examples of each form of reference data.
print('Python List:')
print(str(smList[0:30]))
print('Displaying 30 of ' + str(len(smList)) + ' possible pinyin combinations ' + '\n')
print('——————————————————\n')
print('# Header lists for use in indexing the array/DataFrame')
print('Initials List:')
print(initList)
print('Finals List:')
print(str(finList) + '\n')
print('——————————————————\n')
print('Numpy Array:')
print(str(smAr[0]))
print('Displaying 1 of ' + str(len(smAr)) + ' rows within the array' + '\n')
print('Pandas DataFrame:')
print(smDF.head(3))

Python List:
['a', 'ai', 'ao', 'an', 'ang', 'e', 'ei', 'en', 'eng', 'er', 'o', 'ou', 'yi', 'ya', 'yao', 'ye', 'you', 'yan', 'yang', 'yin', 'ying', 'yong', 'wu', 'wa', 'wo', 'wei', 'wai', 'wan', 'wen', 'wang']
Displaying 30 of 407 possible pinyin combinations 

——————————————————

# Header lists for use in indexing the array/DataFrame
Initials List:
['ø', 'b', 'p', 'm', 'f', 'd', 't', 'n', 'l', 'z', 'c', 's', 'zh', 'ch', 'sh', 'r', 'j', 'q', 'x', 'g', 'k', 'h', 'y', 'w']
Finals List:
['a', 'ai', 'ao', 'an', 'ang', 'e', 'ei', 'en', 'eng', 'er', 'o', 'ong', 'ou', 'i', 'ia', 'iao', 'ie', 'iu', 'ian', 'iang', 'in', 'ing', 'iong', 'u', 'ua', 'uo', 'ue', 'ui', 'uai', 'uan', 'uang', 'un', 'v', 've']

——————————————————

Numpy Array:
[ True  True  True  True  True  True  True  True  True  True  True False
  True False False False False False False False False False False False
 False False False False False False False False False False]
Displaying 1 of 24 rows within the array

Pandas DataFram

## Performing the Tests
To best compare the size and performance speed of each method this test works with two examples: a small two-letter phrase that is an acceptable pinyin term ("ba"), and a longer five-letter term that is not an acceptable pinyin term ("quang"). The %timeit function measures processing time with each method after running 100,000 iterations.

### Methods
1. The Python "in" function: the script searches through the list of acceptable pinyin terms for the supplied example.
2. The examples split into their initial and final pairs, their indexes derived from within the list of initials and finals, and those values are then used to find the appropriate boolean within the NumPy Array.
3. A Pandas DataFrame houses the boolean values, and three Pandas-specific methods are used to slice it:
    1. Traditional slicing method (to be deprecated in future versions): "exampleDF[initial][final]"
    2. The .loc method: "exampleDF.loc[initial][final]"
    3. The .iloc method: "exampleDF.iloc[index of initial within list of initials][index of final within list of finals]"

In [3]:
# Variables used for testing
smEval1 = 'ba'
smInitial1 = 'b'
smFinal1 = 'a'

smEval2 = 'quang'
smInitial2 = 'q'
smFinal2 = 'uang'


# Subject 1: Short, acceptable term
print('# Results of \"ba\" (short, valid):')

# Method 1: Python List
print('Python List: ' + str(sys.getsizeof(smList)))
%timeit -n 100000 smEval1 in smList

# Method 2: Numpy Array
print('\nNumpy array: ' + str(sys.getsizeof(initList) +
                              sys.getsizeof(finList) +
                              sys.getsizeof(smAr)) +
      ', without initials and finals lists: ' + str(sys.getsizeof(smAr)))
%timeit -n 100000 smAr[initList.index(smInitial1),finList.index(smFinal1)] != True

# Method 3: Pandas DataFrame
# 3a: slice
print('\nDataFrame: ' + str(sys.getsizeof(smDF)))
print('slice')
%timeit -n 100000 smDF[smFinal1][smInitial1] != True

# 3b: loc
print('loc')
%timeit -n 100000 smDF.loc[smInitial1,smFinal1] != True

print('iloc')
# 3c: iloc
%timeit -n 100000 smDF.iloc[initList.index(smInitial1),finList.index(smFinal1)] != True



# Subject 2: Longer, unacceptable term
print('\n\n# Results of \"quang\" (longer, invalid):')

# Method 1: Python List
print('Python List: ' + str(sys.getsizeof(smList)))
%timeit -n 100000 smEval2 in smList

# Method 2: Numpy Array
print('\nNumpy array: ' + str(sys.getsizeof(initList) +
                              sys.getsizeof(finList) +
                              sys.getsizeof(smAr)) +
      ', without initials and finals lists: ' + str(sys.getsizeof(smAr)))
%timeit -n 100000 smAr[initList.index(smInitial2),finList.index(smFinal2)] != True

# Method 3: Pandas DataFrame
# 3a: slice
print('\nDataFrame: ' + str(sys.getsizeof(smDF)))
print('slice')
%timeit -n 100000 smDF[smFinal2][smInitial2] != True

# 3b: loc
print('loc')
%timeit -n 100000 smDF.loc[smInitial2,smFinal2] != True

print('iloc')
# 3c: iloc
%timeit -n 100000 smDF.iloc[initList.index(smInitial2),finList.index(smFinal2)] != True

# Results of "ba" (short, valid):
Python List: 3760
399 ns ± 11.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Numpy array: 1672, without initials and finals lists: 928
1.23 µs ± 7.34 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

DataFrame: 2427
slice
8.57 µs ± 39.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
loc
6.99 µs ± 51.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
iloc
8.52 µs ± 97 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


# Results of "quang" (longer, invalid):
Python List: 3760
3.68 µs ± 22.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Numpy array: 1672, without initials and finals lists: 928
1.78 µs ± 34.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

DataFrame: 3067
slice
8.61 µs ± 49.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
loc
7.12 µs ± 37.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
iloc
9.2 µs ± 47.4 ns per loop (mean ± 

## Comparison: Size
The results from the test show that the Pandas DataFrame would initially be the smallest form of data table (2.427kb). However, the DataFrame becomes 25% larger when reaccessed for multiple inquiries due to being loaded into its self-maintained cache (3.067kb). The NumPy array, in addition to the necessary initials and finals lists, takes more space than the initial size of the DataFrame but maintains a small size after repeated uses (2.6kb). The Python list, as expected, takes up the most amount of space due to it listing every possible syllable (3.76kb). Therefore, the NumPy array is the best option assuming the function evaluates multiple queries per use.
## Comparison: Speed
As expected, there is little difference in speed between the NumPy array evaluating the smaller, acceptable syllable and the larger, unacceptable syllable, with only a 0.55 microsecond difference between them. The Pandas DataFrame performed just as consistently, with an average of a 0.28 microsecond difference between the smaller and larger syllables. However, the DataFrame speed as a whole was impractically slower, with an average of 8.17 microseconds per 100,000 iterations, compared to 1.51 microseconds for the NumPy array.

The Python list had the biggest disparity in time with a 3.28 microsecond difference between the smaller and larger syllables, however the incredibly quick .399 microsecond result for the smaller, acceptable phrase initially made the option the most attractive for use. The disadvantage to the method is the entirety of the list being searched for a syllable that does not exist, leading to its 3.68 microsecond result. Future calculations finding the average length of syllables in practical Chinese use could dictate the true average time of the Python list option; however—assuming that the statistical mean will suffice for optimizing this function—the 2.04 microsecond average does not beat that of the NumPy array. Therefore, the array is the best choice as method for this function.

# The sylCount function: a practical explanation
With the NumPy array chosen as the index method for acceptable vs. unacceptable syllables, the sylCount function evaluates the inputted text character-by-character. Computational processes benefit from the consistent syllable structure of the Chinese language because—with very few exceptions—most syllables begin with consonants as their initial and vowels as their final (e.g. "chou:" "ch" as initial, "ou" as final). Therefore, an automated process can quickly establish the initial until it encounters a vowel, accessing the array afterward to determine whether the combination of initial and final is valid.

The validity of syllables is an essential step in the function because not only will it determine whether the current sequence of letters counts as a syllable, but also if the syllable is still valid with additional letters added to it. For example, "tiao." The character-by-character loop within the function sets the initial to "t," and then on its next iteration identifies "i" as the final. "Ti," in itself, is a valid syllable. However, the following character "a" results in "tia," which is not a valid syllable. Searching the array is crucial at this stage because "tiao" is an acceptable syllable, but only if detected during the "tia" evaluation. If no other possibilities exist when evaluating a sequence, then it is safe to assume the current syllable has ended.

There are exceptions to this rule, specifically with the possibility of consonants appearing in the final, e.g., "long." In the pinyin standard, "n," "ng," and "r" (only after following an "e") are all possible consonants in the final. The function, accordingly, must include the ability to evaluate these occurrences and assess whether they are finals, initials, or a combination of both in the case of "ng." Thankfully, these occurrences can be broken down into consistent patterns that are easy to translate into Python.

## The $n$ logic
The tables below illustrate the logic behind the evaluation of $n$. Because $n$ can appear as initial or final, the first and most straightforward approach to identifying its pattern is to look at each possible scenario for any character in pinyin. One character can have nine total possibilities in its placement, depending on whether there is a vowel, consonant, or nothing preceding it, and a vowel, consonant or nothing succeeding it. According to the table, $n$ is only an exception when there is a vowel preceding or succeeding it. All other possibilities are grammatically erroneous, especially if there is a preceding consonant.

In [28]:
from IPython.display import HTML, display
display(HTML("<table style='background-color:black;'><tr><td><img src='img/pinyinPairs.005.png'></td><td><img src='img/pinyinPairs.006.png'></td></tr></table>"))

,


These tables also indicate how the function processes valid possibilities. In the case of *n* with a preceding and succeeding vowel, the syllable count increments by one to account for it being the start of the next syllable. Identifying the character as initial or final is crucial for the function to evaluate the next character correctly.

## The *ng* logic

In [29]:
from IPython.display import HTML, display
display(HTML("<table style='background-color:black;'><tr><td><img src='img/pinyinPairs.003.png'></td><td><img src='img/pinyinPairs.004.png'></td></tr></table>"))

,


For *ng*, the logic is more explicit. The function only identifies the ng pair if there is a preceding vowel since all other possibilities are grammatically incorrect. Also, because *g* is assumed to be the current character in evaluation, the function can quickly identify whether it is the initial of a new syllable or the end of the final based on whether a vowel succeeds it.
## The *er* logic

In [30]:
from IPython.display import HTML, display
display(HTML("<table style='background-color:black;'><tr><td><img src='img/pinyinPairs.001.png'></td><td><img src='img/pinyinPairs.002.png'></td></tr></table>"))

,


The *er* pair has much greater complexity because all nine possibilities are grammatically correct. However, patterns emerge when analyzing their usage in pinyin. If er precedes any character, then the syllable count increments by one to account for the additional syllable. The count also increments by one if er—preceded by a vowel—is succeeded by anything except a vowel. Identifying the current character *r* as initial or final is a simple matter of evaluating whether the succeeding character is a vowel or not.

In [15]:
# Import necessary packages for function
import numpy as np
import csv

## REFERENCE DATA ##
# List of pinyin initials, finals for Numpy evaluation
PY = np.genfromtxt('data/pinyinDF.csv', delimiter=',', dtype=str)
initPYList = list(PY[:,0][1:])
finPYList = list(PY[0][1:])

# List of Wade-Giles initials, finals
WG = np.genfromtxt('data/wadegilesDF.csv', delimiter=',', dtype=str)
initWGList = list(WG[:,0][1:])
finWGList = list(WG[0][1:])

# Import Numpy arrays from CSV
arPY = PY[1:][:,1:] == '1'
arWG = WG[1:][:,1:] == '1'

## MAIN FUNCTION ##
# sylCount definition with parameters:
    # crumbs: show breadcrumbs of NumPy evaluation
    # errorReport: return error messages as strings, NaN for non-errors
    # CBDB: detect duplication of family name within given name
    # wg: evaluate in Wade-Giles standard
def sylCount(text, crumbs=False, errorReport=False, CBDB=False, wg=False):
    '''Estimate the number of Chinese characters based on inputted romanized text.'''
    
    ## REFERENCE VARIABLES ##
    # Lists/array for reference (pinyin set as default)
    vowel = ['a','e','i','o',
             'u','ü','v'] # for detecting switch from initial to final
    initList = initPYList # list of initials
    finList = finPYList # list of finals
    ar = arPY # array of boolean values
    
    ## DATA EVALUATION ##
    # Massage data for evaluation
        # Replace ideographic symbols
        # Strip trailing/leading spaces
        # Convert to lowercase
    # If the process fails, the evaluated text is null
    replSym = {'—':'-',
               '–':'-',
               '\'':'’',
               '\u3000':' '}
    try:
        for i, n in replSym.items():
            text = text.replace(i,n)
        text = text.strip().lower()
        evalText = text # used for detection of invalid characters
        for n in replSym.values():
            evalText = evalText.replace(n,'').strip().lower()
    except:
        errorMsg = 'Error: null value'
        if crumbs: print(errorMsg)
        if errorReport: return errorMsg
        return [0]
    
    # If invalid characters are detected
    if not evalText.isalpha():
        errorMsg = 'Error: invalid non-alpha characters in name (' + evalText + ')'
        if crumbs: print(errorMsg)
        if errorReport: return errorMsg
        return [0]        
    
    # If hyphens or Wade-Giles are detected, change reference variables to WG:
    if '-' in text or wg:
        wg = True
        initList = initWGList
        finList = finWGList
        ar = arWG
        if crumbs: print('Switching to Wade-Giles: hyphen detected')
    
    # CBDB testing purposes ONLY: if the family name is duplicated in the given name
    if CBDB:
        dupTest = text.split()
        if len(dupTest) > 1:
            dupTestLen = [len(dupTest[0]), len(dupTest[1])]
            if (dupTest[0] == dupTest[1][0:dupTestLen[0]] and 
                dupTestLen[1] > dupTestLen[0] and 
                dupTest[1][len(dupTest[0])] != '\''):
                errorMsg = 'Error: duplicate family name in given name'
                if crumbs: print(errorMsg)
                if errorReport: return errorMsg 
                return [0]

    ## MAIN EVALUATION VARIABLES ##
    # Variables used for evaluation
    initial = ''
    final = ''
    newSyl = False
    syl = 1
    sylCount = []
    exFin = ''
    
    if crumbs: print('# Testing ' + text + ' #')
    
    ## MAIN EVALUATION ##
    # Cycle through each character in text
    for i,n in enumerate(text):

        ## SPECIAL CHARACTERS ##
        # Current character: Wade-Giles hyphen - new syllable
        if n == '-' and wg:
            step = '0. new syl'
            notes = 'hyphen'
            newSyl = True
        
        # Current character: space - new word
        elif n == ' ':
            step = '0. new word'
            
            # If double-spaced, skip to next character
            if text[i+1] == ' ':
                notes = 'discard extra space'
                
            # Otherwise, save syllable count to list and start next word
            else:
                sylCount.append(syl)
                newSyl = True
                notes = 'space'

        ## FINALS ##
        # If final has already been set, regardless of current character
        elif final:
            step = 'b. ' + initial + '|' + final + n
            
            # If current character is an apostrophe (as with Wade-Giles)
            if n == '’':
                newSyl = True
                notes = 'new syl - apostrophe'
            
            # If 'er,' 'ng,' or 'n'
            elif (final[-1] == 'e' and n == 'r' or 
                final[-1] == 'n'):
                notes = final[-1] + n
                
                # If there are more characters ahead, mark as new syllable
                if i != len(text) - 1 and text[i+1] != ' ':
                    newSyl = True
                    exFin = final[-1] + n
                    notes += ' - not \"' + exFin + '\" final'
                    
                # Else if there were other vowels before 'er,' add a syllable
                elif (notes == 'er' and len(final) > 1) or n in vowel:
                    syl += 1
                    notes += ' - extra syl at end'

            # Check array with current initial + final combo
            else:
                notes = 'fin'
                
                # If there's an error, consider it as not a valid syllable
                try:
                    result = ar[
                        initList.index(initial),
                        finList.index(final + n)
                    ]
                    notes += ' - array lookup ' + initial + final + n + ': ' + str(result)
                except:
                    result = False
                    notes += ' - array error'

                # If not a valid syllable, is it part of a longer syllable?
                if not result:
                    listFins = list(i for i in finList if i.startswith(final+n))
                    if not listFins or not any(
                        i for i in listFins if ar[
                            initList.index(initial),
                            finList.index(i)
                        ]
                    ):    # It is!
                        newSyl = True
                        notes += ' - new syl'
                    else: # It isn't.
                        final += n
                        notes += ' - definitely more'
                
                # Otherwise, the syllable is valid
                else:
                    final += n
                    notes += ' - maybe more'
          
        ## INITIALS ##
        # If there is no final set (then must be initial)
        else:
            step = 'a. ' + initial + n
            notes = 'init'
            
            # If character is not a vowel
            if n not in vowel:
                
                # Check if character is (or is part of) a valid initial
                if not any(i for i in initList if i.startswith(initial + n)):
                    errorMsg = 'Error: ' + initial + n + ' - quitting, invalid initial'
                    if crumbs: print(errorMsg)
                    if errorReport: return errorMsg
                    return [0]
                initial += n
                notes += ' - cons'
            
            # Or if the vowel is the first character of the syllable
            elif not initial:
                initial = 'ø'
                final = n
                notes += ' - starting vowel'
                
            # Then the vowel is the first character of the final
            else:
                final = n
                notes += ' - vowel'
                
        ## DEBUG NOTES ##
        # Printed if crumbs parameter is true
        if crumbs: print(step + ': ' + notes)
        
        ## NEW SYLLABLE ##
        # Set up next initial and/or final based on prior evaluation above
        if newSyl:
            
            # If current letter is a vowel
            if n in vowel:
                
                # If final was being evaluated for 'n' (e.g. 'ch|iN-A|ng')
                if exFin:
                    initial = 'n'
                
                # Otherwise it's simply a new syllable (e.g. 'ch|eng-A|ng')
                else:
                    initial = 'ø'
                syl += 1
                final = n
            
            # Or it's anything but a space (consonant, hyphen, apostrophe)
            elif n != ' ':
                
                # If character is break symbol or next character is next syllable
                if (
                    (exFin and text[i+1] not in vowel) or
                    (n == '’' or n == '-')
                ):
                    initial = ''
                    
                    # If it's the 'er' situation with a vowel before it
                    if exFin == 'er' and len(final) > 1:
                        syl+= 1
                
                # Then current character is the start of next syllable
                else:
                    initial = n
                exFin = False
                final = ''
                syl += 1
            
            # Then it is a space (new word)
            else:
                initial = ''
                final = ''
                syl = 1
            if crumbs: print('new syl: ' + str(syl))
            newSyl = False
    
    # Append syllable count to list after end of text evaulation
    sylCount.append(syl)
    
    # Return a null value if the errorReport parameter is true
    if errorReport:
        return np.nan
    
    # Otherwise return the list of syllable counts
    else:
        return sylCount

In [22]:
import pandas as pd

# Import CBDB data
names = pd.read_csv('data/allCBDBnames.csv', index_col=0,
                    names=['EngName','ChiName'], header=0)

# Run sylCount function on 'EngName' column
names['syl'] = names['EngName'].apply(sylCount, CBDB=True)

# Add 'check' column to validate results
def tryAdd(x):
    try:
        return sum(x)
    except:
        return
names['check'] = names['ChiName'].str.len() == names['syl'].apply(tryAdd)

# Filter results to show false negatives
names[~names['check']]

,EngName,ChiName,syl,check
26,Chen Ji(2),陳機,[0],False
31,Chen Jian(2),陳戩,[0],False
32,Chen Zhi(5),陳致,[0],False
37,Chen Jing(5),陳靖,[0],False
41,Chen Zhu(2),陳鑄,[0],False
42,Chen Ju(3),陳鉅,[0],False
48,Chen Xiang(2),陳相,[0],False
54,Chen Hong(2),陳洪,[0],False
58,Chen Ruxi(2),陳汝錫,[0],False
83,Chen Shu(2),陳恕,[0],False


In [17]:
sum(sylCount('Hsüeh chieh t\'ai tou ming jen pi hsia te Ts\'ai Yüan-p\'ei Ts\'ai Yüan-p\'ei pi hsia te ming jen Yüan Chin pien', crumbs=True))

Switching to Wade-Giles: hyphen detected
# Testing hsüeh chieh t’ai tou ming jen pi hsia te ts’ai yüan-p’ei ts’ai yüan-p’ei pi hsia te ming jen yüan chin pien #
a. h: init - cons
a. hs: init - cons
a. hsü: init - vowel
b. hs|üe: fin - array error - definitely more
b. hs|üeh: fin - array lookup hsüeh: True - maybe more
0. new word: space
new syl: 1
a. c: init - cons
a. ch: init - cons
a. chi: init - vowel
b. ch|ie: fin - array error - definitely more
b. ch|ieh: fin - array lookup chieh: True - maybe more
0. new word: space
new syl: 1
a. t: init - cons
a. t’: init - cons
a. t’a: init - vowel
b. t’|ai: fin - array lookup t’ai: True - maybe more
0. new word: space
new syl: 1
a. t: init - cons
a. to: init - vowel
b. t|ou: fin - array lookup tou: True - maybe more
0. new word: space
new syl: 1
a. m: init - cons
a. mi: init - vowel
b. m|in: fin - array lookup min: True - maybe more
b. m|ing: ng
0. new word: space
new syl: 1
a. j: init - cons
a. je: init - vowel
b. j|en: fin - array lookup jen

23

In [19]:
sylCount('Hai wai chi zi Nanyang xie ying Huang Xing xian sheng juan zeng Zhongguo Hua qiao li shi bo wu guan wen wu jing pin tu lu', crumbs=True)

# Testing hai wai chi zi nanyang xie ying huang xing xian sheng juan zeng zhongguo hua qiao li shi bo wu guan wen wu jing pin tu lu #
a. h: init - cons
a. ha: init - vowel
b. h|ai: fin - array lookup hai: True - maybe more
0. new word: space
new syl: 1
a. w: init - cons
a. wa: init - vowel
b. w|ai: fin - array lookup wai: True - maybe more
0. new word: space
new syl: 1
a. c: init - cons
a. ch: init - cons
a. chi: init - vowel
0. new word: space
new syl: 1
a. z: init - cons
a. zi: init - vowel
0. new word: space
new syl: 1
a. n: init - cons
a. na: init - vowel
b. n|an: fin - array lookup nan: True - maybe more
b. n|any: ny - not "ny" final
new syl: 2
a. ya: init - vowel
b. y|an: fin - array lookup yan: True - maybe more
b. y|ang: ng
0. new word: space
new syl: 1
a. x: init - cons
a. xi: init - vowel
b. x|ie: fin - array lookup xie: True - maybe more
0. new word: space
new syl: 1
a. y: init - cons
a. yi: init - vowel
b. y|in: fin - array lookup yin: True - maybe more
b. y|ing: ng
0. new 

[1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]